In [13]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [80]:
file='./data/1.xlsx'
df1=pd.read_excel(file,sheet_name='四个自变量99-17',
keep_default_na=False,
    dtype={
        'COMPANY': str,
        'YEAR': str,
        'ID': str,
    })
df2=pd.read_excel(file,sheet_name='违规信息黑名单94-20',keep_default_na=False,
    dtype={
        'COMPANY': str,
        'YEAR': str,
        'YEARS': str,
        'ID': str
    })

df1.drop(["有格式版本"],axis=1,inplace=True)
# df1.set_index('ID')
# df2.set_index('ID')
# df1[df1["DEPI"].isnull()]=None
# df1=df1.dropna().reset_index(drop=True)
# df2=df2.dropna().reset_index(drop=True)
df1.head()
# df2.head()

,COMPANY,DATE,YEAR,ID,DEPI,GAIN,LOSS,TATA1,TATA2,CHCS,...,GMIII,SGAI,CHROA,AQI,LVGI,DSRI,SGI,SOFTAS,CHINV,CHREC
0,000001,2014-12-31,2014,20140000001,0.973702,1.984100e+10,0.0,-9.147210e-07,0.000440,0.087320,...,1.0,NaN,0.001003,0.892846,0.999316,0.970355,1.377101,0.998257,NaN,0.031616
1,000001,2013-12-31,2013,20130000001,0.948619,1.516600e+10,0.0,2.666105e-04,0.000696,-0.803567,...,1.0,NaN,-0.000725,0.887073,0.993176,1.466035,1.318226,0.998047,NaN,0.052890
2,000001,2012-12-31,2012,20120000001,0.877270,1.338522e+10,0.0,-9.296631e-04,-0.000479,3.187515,...,1.0,NaN,-0.001034,0.785867,1.007583,0.402320,1.337744,0.997799,NaN,-0.059426
3,000001,2011-12-31,2011,20110000001,1.004569,1.017927e+10,0.0,3.980625e-04,0.000758,-4.228793,...,1.0,NaN,0.000910,0.851472,0.985056,3.452926,1.656281,0.997199,NaN,0.153211
4,000001,2010-12-31,2010,20100000001,1.155798,6.143737e+09,0.0,-2.697994e-05,0.000207,1.169672,...,1.0,NaN,0.000085,1.127889,0.988782,0.745035,1.228627,0.996710,NaN,-0.004532


In [84]:
# print(df1.isnull().sum())
print(df1.shape)
columns=list(df1.columns)
columns.remove("DEPI")
df3=df1.copy()
for col in columns:
    df3=df3[~df3[col].isnull()]
# print(a)
print(df3.shape)
# print(df2.info())
# df1.iloc[31520:]
# df1[df1["GAIN"] > 0.5]


(35800, 22)
(30510, 22)


In [85]:
def cal_cheat_ids():
    cheat_ids=[]
    for index, row in df2.iterrows():
        COMPANY=row['COMPANY']
        YEARS=row['YEARS'].strip(" ").split(",")
        for YEAR in YEARS:
            cheat_ids.append(str(int(YEAR)*10000000+int(COMPANY)))
    return cheat_ids

#  show the first 50 items
cheat_ids=cal_cheat_ids()
print(len(cheat_ids))
print(cheat_ids[:50])


1009
['20120000007', '20130000007', '20140000007', '20150000010', '20160000010', '20010000013', '20140000020', '20150000020', '20110000033', '20120000033', '20130000033', '20140000033', '20030000034', '20040000034', '20040000035', '19990000037', '20000000037', '20010000037', '20020000037', '20030000038', '20040000038', '20060000040', '20070000040', '20080000040', '20090000040', '20100000040', '20020000156', '20030000156', '20040000156', '20070000403', '20080000403', '20090000403', '20100000403', '20110000403', '20120000403', '20130000403', '20120000408', '20130000408', '20170000408', '20180000408', '20190000408', '20000000409', '20010000409', '20020000409', '20030000409', '20040000409', '20030000421', '20040000421', '20030000498', '20040000498']


In [86]:
cheat_ids=cal_cheat_ids()

# df3=pd.merge(df1,df2[["ID","CHEAT"]],how="left",on="ID")
df3["CHEAT"]=None

df_0=df3[~df3["ID"].isin(cheat_ids)]
df_0["CHEAT"]=0

df_1=df3[df3["ID"].isin(cheat_ids)]
df_1["CHEAT"]=1

df3=pd.concat([df_0,df_1]).reset_index(drop=True)

print(df3.isnull().any())
df3.to_csv("./data/1.csv",index=False)

# with tqdm(total=df3.shape[0]) as pbar:
#     for index, row in df3.iterrows():
#         # if index and (index%20)==0:
#             # print(index)
#             # pbar.update(20)
#         # if row["ID"]== "20120000007":
#         #     print(row["ID"] in cheat_ids)
#         if row["ID"] in cheat_ids:
#             df3["CHEAT"].iloc[index]=1
#             cheat_ids.remove(row["ID"])
#         else:
#             df3["CHEAT"].iloc[index]=0
#         pbar.update(1)

# print(df3.isnull().any())
# df3.to_csv("./data/1.csv",index=False)

COMPANY    False
DATE       False
YEAR       False
ID         False
DEPI        True
GAIN       False
LOSS       False
TATA1      False
TATA2      False
CHCS       False
OTHREC     False
GMI        False
GMIII      False
SGAI       False
CHROA      False
AQI        False
LVGI       False
DSRI       False
SGI        False
SOFTAS     False
CHINV      False
CHREC      False
CHEAT      False
dtype: bool


In [87]:
print(df3[df3["CHEAT"]==1].shape)
print(df3[df3["CHEAT"]==0].shape)

(689, 23)
(29821, 23)
